In [1]:
#decoding the data
import numpy as np
import tensorflow as tf
import helpers

import os



SEPCTURAL_SAMPLES = 40

#FEATURE_DIM = SEPCTURAL_SAMPLES*6*2   # This data set = 30
FEATURE_DIM=30



#INTER_DIM = 120  #60
INTER_DIM=121


#OUT_DIM = 6#len(idDict)  #66
OUT_DIM = 66


#WIDE = 20  #time step  #40
WIDE=121





BATCH_SIZE = 16
TOTAL_ITER_NUM = 1000000000

In [2]:
import pdb
def read_audio_csv(filename_queue):
	reader = tf.TextLineReader()
	key, value = reader.read(filename_queue)

	defaultVal = [[0.] for idx in range(WIDE*FEATURE_DIM + OUT_DIM)]

	fileData = tf.decode_csv(value, record_defaults=defaultVal)

	features = fileData[:WIDE*FEATURE_DIM]
  
	features = tf.reshape(features, [WIDE, FEATURE_DIM])
	labels = fileData[WIDE*FEATURE_DIM:] 

	return features, labels

def input_pipeline(filenames, batch_size, shuffle_sample=True, num_epochs=None):
	filename_queue = tf.train.string_input_producer(filenames, shuffle=shuffle_sample)

	# filename_queue = tf.train.string_input_producer(filenames, num_epochs=TOTAL_ITER_NUM*EVAL_ITER_NUM*10000000, shuffle=shuffle_sample)
	example, label = read_audio_csv(filename_queue)
	min_after_dequeue = 2000#int(0.4*len(csvFileList)) #1000
	capacity = min_after_dequeue + 3 * batch_size
	if shuffle_sample:
		example_batch, label_batch = tf.train.shuffle_batch(
			[example, label], batch_size=batch_size, num_threads=16, capacity=capacity,
			min_after_dequeue=min_after_dequeue)
	else:
		example_batch, label_batch = tf.train.batch(
			[example, label], batch_size=batch_size, num_threads=16)
	return example_batch, label_batch

In [3]:
csvFileList = []

#csvDataFolder1 = os.path.join('N_Z')
csvDataFolder1 = os.path.join('Data')

print(csvDataFolder1)
orgCsvFileList = os.listdir(csvDataFolder1)

len(orgCsvFileList)


Data


1186

In [4]:
for csvFile in orgCsvFileList:
	if csvFile.endswith('.csv'):
		csvFileList.append(os.path.join(csvDataFolder1, csvFile))

In [5]:
len(csvFileList)

1186

In [6]:
batch_size=16
batch_feature, batch_label = input_pipeline(csvFileList, batch_size)
#print(batch_feature)
#pdb.set_trace() 
#print(batch_label)

In [7]:
sensor_inputs = tf.expand_dims(batch_feature, axis=3)

In [8]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 121, 30, 1) dtype=float32>

In [9]:
output_keep_prob=0.5
train=True

gru_cell1 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell1 = tf.contrib.rnn.DropoutWrapper(gru_cell1, output_keep_prob=0.5)

gru_cell2 = tf.contrib.rnn.GRUCell(128)
if train:
    gru_cell2 = tf.contrib.rnn.DropoutWrapper(gru_cell2, output_keep_prob=0.5)

In [10]:
used = tf.sign(tf.reduce_max(tf.abs(batch_feature), reduction_indices=2))

In [11]:
#print(batch_feature)

In [12]:
length = tf.reduce_sum(used, reduction_indices=1) #(BATCH_SIZE)

In [13]:
length

<tf.Tensor 'Sum:0' shape=(16,) dtype=float32>

In [14]:
length = tf.cast(length, tf.int64)
length

<tf.Tensor 'Cast:0' shape=(16,) dtype=int64>

In [15]:
sensor_inputs

<tf.Tensor 'ExpandDims:0' shape=(16, 121, 30, 1) dtype=float32>

In [16]:
#cell = tf.contrib.rnn.MultiRNNCell([gru_cell1, gru_cell2])
cell = tf.contrib.rnn.MultiRNNCell([gru_cell1,gru_cell2])

In [17]:
init_state = cell.zero_state(BATCH_SIZE, tf.float32)

In [18]:
cell_output, final_stateTuple = tf.nn.dynamic_rnn(cell, batch_feature, sequence_length=length, initial_state=init_state, time_major=False)

In [19]:
cell_output

<tf.Tensor 'rnn/transpose_1:0' shape=(16, 121, 128) dtype=float32>

In [20]:
inputs=batch_feature
mask = tf.sign(tf.reduce_max(tf.abs(inputs), reduction_indices=2, keepdims=True))
print(mask)
mask = tf.tile(mask, [1,1,128]) # (BATCH_SIZE, WIDE, INTER_DIM)
avgNum = tf.reduce_sum(mask, reduction_indices=1) #(BATCH_SIZE, INTER_DIM)
print mask

Tensor("Sign_1:0", shape=(16, 121, 1), dtype=float32)
Tensor("Tile:0", shape=(16, 121, 128), dtype=float32)


In [21]:
print(mask)
print(avgNum )

Tensor("Tile:0", shape=(16, 121, 128), dtype=float32)
Tensor("Sum_1:0", shape=(16, 128), dtype=float32)


In [22]:
layers = tf.contrib.layers 
sum_cell_out = tf.reduce_sum(cell_output*mask, axis=1, keep_dims=False)
avg_cell_out = sum_cell_out/avgNum

logits = layers.fully_connected(avg_cell_out, OUT_DIM, activation_fn=None, scope='output')


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [23]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()

regularizers = 0.
for var in t_vars:
	regularizers += tf.nn.l2_loss(var)
loss += 5e-4 * regularizers


discOptimizer = tf.train.AdamOptimizer(
		learning_rate=1e-4, 
		beta1=0.5,
		beta2=0.9
	).minimize(loss, var_list=t_vars)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [ ]:
tf.initialize_all_variables()

saver = tf.train.Saver()

Train_Acc=np.mean(batch_label==predict)
Train_Loss=loss

T_accuracy_summary = tf.summary.scalar("Training_Accuracy",Train_Acc)
T_loss_summary = tf.summary.scalar("Training_Loss",Train_Loss)
summary_op = tf.summary.merge_all()

lossz=[]
accz=[]
epochz=[]

with tf.Session() as sess:
	tf.train.start_queue_runners()
	sess.run(tf.local_variables_initializer())
	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)
	checkpoint_dir='/home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master'
	ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
	writer = tf.summary.FileWriter('/home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/summery/', graph=sess.graph)

	if ckpt and ckpt.model_checkpoint_path:
		print(ckpt.model_checkpoint_path)
		if os.path.isabs(ckpt.model_checkpoint_path):
			saver.restore(sess, ckpt.model_checkpoint_path)
		else:
			print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzgdgdgd")
			saver.restore(sess, os.path.join(checkpoint_dir,ckpt.model_checkpoint_path))
		global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		print('Successfully loaded model from %s at step=%s.' %(ckpt.model_checkpoint_path, global_step))
    

	for iteration in xrange(60000):
		print("Iteration NUmber :", iteration)        
		_, sumz,lossV, _trainY, _predict = sess.run([discOptimizer,summary_op , loss, batch_label, predict])
		writer.add_summary(sumz,iteration)
     
		print(lossV)       
		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)   

		coord.request_stop()
		coord.join(threads)
 

            
        
		if iteration % 20000 == 0:
			save_path = saver.save(sess, "/home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check_n/",global_step=iteration)
			print("Model saved in path: %s" % save_path)
            


Instructions for updating:
Use `tf.global_variables_initializer` instead.
('Iteration NUmber :', 0)
4.810129
Model saved in path: /home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master/check_n/-0
('Iteration NUmber :', 1)
4.842894
('Iteration NUmber :', 2)
4.886341
('Iteration NUmber :', 3)
4.6893406
('Iteration NUmber :', 4)
4.662804
('Iteration NUmber :', 5)
4.647471
('Iteration NUmber :', 6)
4.559527
('Iteration NUmber :', 7)
4.7118692
('Iteration NUmber :', 8)
4.7171016
('Iteration NUmber :', 9)
4.5984263
('Iteration NUmber :', 10)
4.491505
('Iteration NUmber :', 11)
4.448794
('Iteration NUmber :', 12)
4.5106525
('Iteration NUmber :', 13)
4.545127
('Iteration NUmber :', 14)
4.4791865
('Iteration NUmber :', 15)
4.402865
('Iteration NUmber :', 16)
4.3605957
('Iteration NUmber :', 17)
4.529333
('Iteration NUmber :', 18)
4.516962
('Iteration NUmber :', 19)
4.558812
('Iteration NUmber :', 20)
4.3442845
('Iteration NUmber :', 21)
4.4380784
('Iteration NUmb

2.9640412
('Iteration NUmber :', 210)
2.740271
('Iteration NUmber :', 211)
3.005742
('Iteration NUmber :', 212)
3.0713887
('Iteration NUmber :', 213)
3.6978295
('Iteration NUmber :', 214)
2.8309045
('Iteration NUmber :', 215)
3.6596878
('Iteration NUmber :', 216)
3.4835393
('Iteration NUmber :', 217)
2.8681774
('Iteration NUmber :', 218)
3.2537122
('Iteration NUmber :', 219)
2.8264215
('Iteration NUmber :', 220)
3.4956286
('Iteration NUmber :', 221)
2.8172648
('Iteration NUmber :', 222)
3.522114
('Iteration NUmber :', 223)
3.6181993
('Iteration NUmber :', 224)
3.5447273
('Iteration NUmber :', 225)
2.9139717
('Iteration NUmber :', 226)
3.398772
('Iteration NUmber :', 227)
2.9897528
('Iteration NUmber :', 228)
3.8971221
('Iteration NUmber :', 229)
2.5498044
('Iteration NUmber :', 230)
3.2939534
('Iteration NUmber :', 231)
3.683815
('Iteration NUmber :', 232)
2.458229
('Iteration NUmber :', 233)
3.1779578
('Iteration NUmber :', 234)
3.2259843
('Iteration NUmber :', 235)
3.3919687
('Iterat

3.0060077
('Iteration NUmber :', 427)
2.394029
('Iteration NUmber :', 428)
2.4669838
('Iteration NUmber :', 429)
3.5389984
('Iteration NUmber :', 430)
2.922394
('Iteration NUmber :', 431)
3.3706613
('Iteration NUmber :', 432)
2.8110669
('Iteration NUmber :', 433)
2.5994334
('Iteration NUmber :', 434)
3.439248
('Iteration NUmber :', 435)
3.1987648
('Iteration NUmber :', 436)
2.6502388
('Iteration NUmber :', 437)
2.327706
('Iteration NUmber :', 438)
3.1258464
('Iteration NUmber :', 439)
2.577686
('Iteration NUmber :', 440)
2.8709302
('Iteration NUmber :', 441)
2.9386468
('Iteration NUmber :', 442)
2.7243588
('Iteration NUmber :', 443)
2.409133
('Iteration NUmber :', 444)
3.0360792
('Iteration NUmber :', 445)
2.3906422
('Iteration NUmber :', 446)
2.629753
('Iteration NUmber :', 447)
2.3533487
('Iteration NUmber :', 448)
2.4239724
('Iteration NUmber :', 449)
2.213168
('Iteration NUmber :', 450)
2.872563
('Iteration NUmber :', 451)
2.7476442
('Iteration NUmber :', 452)
2.0124056
('Iteration

3.4670043
('Iteration NUmber :', 645)
3.052192
('Iteration NUmber :', 646)
2.1595993
('Iteration NUmber :', 647)
2.6406395
('Iteration NUmber :', 648)
1.7483284
('Iteration NUmber :', 649)
2.5294304
('Iteration NUmber :', 650)
1.9640734
('Iteration NUmber :', 651)
1.9950485
('Iteration NUmber :', 652)
2.5356567
('Iteration NUmber :', 653)
2.1843228
('Iteration NUmber :', 654)
2.0098867
('Iteration NUmber :', 655)
2.4905992
('Iteration NUmber :', 656)
2.930445
('Iteration NUmber :', 657)
2.7025251
('Iteration NUmber :', 658)
1.7345655
('Iteration NUmber :', 659)
1.8467032
('Iteration NUmber :', 660)
2.7110095
('Iteration NUmber :', 661)
1.8910275
('Iteration NUmber :', 662)
2.2440133
('Iteration NUmber :', 663)
2.3417892
('Iteration NUmber :', 664)
2.4205012
('Iteration NUmber :', 665)
2.5019913
('Iteration NUmber :', 666)
2.264323
('Iteration NUmber :', 667)
1.8516948
('Iteration NUmber :', 668)
2.318756
('Iteration NUmber :', 669)
2.5065117
('Iteration NUmber :', 670)
2.0667868
('Iter

3.011263
('Iteration NUmber :', 862)
3.375582
('Iteration NUmber :', 863)
2.3466895
('Iteration NUmber :', 864)
2.1755905
('Iteration NUmber :', 865)
2.7088609
('Iteration NUmber :', 866)
2.7216291
('Iteration NUmber :', 867)
2.1794572
('Iteration NUmber :', 868)
3.1058228
('Iteration NUmber :', 869)
1.9641773
('Iteration NUmber :', 870)
2.027991
('Iteration NUmber :', 871)
2.6008706
('Iteration NUmber :', 872)
2.6088147
('Iteration NUmber :', 873)
2.2165031
('Iteration NUmber :', 874)
2.6494772
('Iteration NUmber :', 875)
2.2103026
('Iteration NUmber :', 876)
2.4058056
('Iteration NUmber :', 877)
2.4456797
('Iteration NUmber :', 878)
2.052838
('Iteration NUmber :', 879)
1.8974613
('Iteration NUmber :', 880)
2.429981
('Iteration NUmber :', 881)
2.0112474
('Iteration NUmber :', 882)
2.5120466
('Iteration NUmber :', 883)
2.5560982
('Iteration NUmber :', 884)
2.330466
('Iteration NUmber :', 885)
2.6146104
('Iteration NUmber :', 886)
2.36693
('Iteration NUmber :', 887)
1.9671199
('Iteratio

1.7273972
('Iteration NUmber :', 1077)
2.1844954
('Iteration NUmber :', 1078)
1.5687463
('Iteration NUmber :', 1079)
2.275564
('Iteration NUmber :', 1080)
2.03798
('Iteration NUmber :', 1081)
2.8416371
('Iteration NUmber :', 1082)
2.0733943
('Iteration NUmber :', 1083)
1.9272897
('Iteration NUmber :', 1084)
2.7854714
('Iteration NUmber :', 1085)
1.453172
('Iteration NUmber :', 1086)
2.3026757
('Iteration NUmber :', 1087)
2.0032332
('Iteration NUmber :', 1088)
2.4363198
('Iteration NUmber :', 1089)
1.8682553
('Iteration NUmber :', 1090)
2.3511982
('Iteration NUmber :', 1091)
1.9127502
('Iteration NUmber :', 1092)
2.6323433
('Iteration NUmber :', 1093)
2.006493
('Iteration NUmber :', 1094)
2.827298
('Iteration NUmber :', 1095)
3.0897484
('Iteration NUmber :', 1096)
2.4340675
('Iteration NUmber :', 1097)
2.7948735
('Iteration NUmber :', 1098)
2.5169792
('Iteration NUmber :', 1099)
1.783776
('Iteration NUmber :', 1100)
2.00561
('Iteration NUmber :', 1101)
1.9082648
('Iteration NUmber :', 1

1.6314646
('Iteration NUmber :', 1289)
2.95319
('Iteration NUmber :', 1290)
2.653304
('Iteration NUmber :', 1291)
2.4931643
('Iteration NUmber :', 1292)
2.6351123
('Iteration NUmber :', 1293)
1.9800256
('Iteration NUmber :', 1294)
2.3785534
('Iteration NUmber :', 1295)
2.9430602
('Iteration NUmber :', 1296)
1.9193366
('Iteration NUmber :', 1297)
2.23996
('Iteration NUmber :', 1298)
1.7794498
('Iteration NUmber :', 1299)
2.3325763
('Iteration NUmber :', 1300)
1.6624625
('Iteration NUmber :', 1301)
1.8233073
('Iteration NUmber :', 1302)
2.2271817
('Iteration NUmber :', 1303)
1.872327
('Iteration NUmber :', 1304)
1.8097122
('Iteration NUmber :', 1305)
1.5850048
('Iteration NUmber :', 1306)
2.0681188
('Iteration NUmber :', 1307)
2.1147501
('Iteration NUmber :', 1308)
2.0958931
('Iteration NUmber :', 1309)
2.3324332
('Iteration NUmber :', 1310)
2.2206452
('Iteration NUmber :', 1311)
2.2037852
('Iteration NUmber :', 1312)
1.9264556
('Iteration NUmber :', 1313)
1.321538
('Iteration NUmber :',

2.2253845
('Iteration NUmber :', 1501)
1.9017051
('Iteration NUmber :', 1502)
2.1720362
('Iteration NUmber :', 1503)
2.438373
('Iteration NUmber :', 1504)
1.9951319
('Iteration NUmber :', 1505)
2.1963997
('Iteration NUmber :', 1506)
2.3510113
('Iteration NUmber :', 1507)
2.4282873
('Iteration NUmber :', 1508)
2.1955376
('Iteration NUmber :', 1509)
1.6801445
('Iteration NUmber :', 1510)
2.587971
('Iteration NUmber :', 1511)
1.6332301
('Iteration NUmber :', 1512)
1.547574
('Iteration NUmber :', 1513)
2.1847801
('Iteration NUmber :', 1514)
2.2561395
('Iteration NUmber :', 1515)
2.1708145
('Iteration NUmber :', 1516)
1.9076937
('Iteration NUmber :', 1517)
2.115348
('Iteration NUmber :', 1518)
1.8372517
('Iteration NUmber :', 1519)
1.6207824
('Iteration NUmber :', 1520)
1.6641471
('Iteration NUmber :', 1521)
1.7009239
('Iteration NUmber :', 1522)
2.0648067
('Iteration NUmber :', 1523)
1.843214
('Iteration NUmber :', 1524)
2.1633875
('Iteration NUmber :', 1525)
1.5140868
('Iteration NUmber :

In [ ]:
'
#		if iteration % 200 == 0:
#			lossz.append(lossV) 
#			epochz.append(iteration)
#			accz.append(_accuracy)
'''

In [ ]:
import matplotlib.pyplot as plt

radius = [1.0, 2.0, 3.0, 4.0]
area = [3.14159, 12.56636, 28.27431, 50.26544]

plt.plot(radius, area)
plt.show()

In [ ]:
predict = tf.argmax(logits, axis=1)
predict
batchLoss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=batch_label)
loss = tf.reduce_mean(batchLoss)
t_vars = tf.trainable_variables()



#tf.initialize_all_variables()
checkpoint_dir='/home/dl/Documents/Shamane/Shamane Research - Repo/tensorflow-seq2seq-tutorials-master'
saver = tf.train.Saver()
tf.initialize_all_variables()
with tf.Session() as sess:
	tf.train.start_queue_runners()
#	sess.run(tf.local_variables_initializer())
#	tf.global_variables_initializer().run()

	coord = tf.train.Coordinator()
	threads = tf.train.start_queue_runners(coord=coord)    
	ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    
	if ckpt and ckpt.model_checkpoint_path:
		print(ckpt.model_checkpoint_path)
		if os.path.isabs(ckpt.model_checkpoint_path):
			saver.restore(sess, ckpt.model_checkpoint_path)
		else:
			print("zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzgdgdgd")
			saver.restore(sess, os.path.join(checkpoint_dir,ckpt.model_checkpoint_path))
		global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
		print('Successfully loaded model from %s at step=%s.' %(ckpt.model_checkpoint_path, global_step))



	for iteration in xrange(100000):
		print("Iteration NUmber :", iteration)        
		_trainY, _predict = sess.run([batch_label, predict])
        
		        

		_label = np.argmax(_trainY, axis=1)
		_accuracy = np.mean(_label == _predict)        
		print(_accuracy)
		coord.request_stop()
		coord.join(threads)
        

            